## 获取模型的总参数量，总计算量，每1类操作的次数

In [ ]:
#model每1层的参数信息元组的列表
summary = []
#每1层对应的具名元组
ModuleDetails = namedtuple("Layer", ["name", "input_size", "output_size", "num_parameters", "multiply_adds"])
#每1层对应的hook列表
hooks = []
#k-v:网络层类名-出现的次数
layer_instances = {}

## 计算每1类操作的次数

In [ ]:
#网络层module的类名
class_name = str(module.__class__.__name__)

instance_index = 1
if class_name not in layer_instances:
    layer_instances[class_name] = instance_index
else:
    instance_index = layer_instances[class_name] + 1
    layer_instances[class_name] = instance_index

#网络层名字=网络层类名_出现的次数
layer_name = class_name + "_" + str(instance_index)

## 计算某1层的参数量

In [ ]:
#参数数量
params = 0
#卷积层/Batch norm层/Linear层
if class_name.find("Conv") != -1 or class_name.find("BatchNorm") != -1 or \
   class_name.find("Linear") != -1:
    #params：module.parameters()参数的数量
    for param_ in module.parameters():
        params += param_.view(-1).size(0)

## 计算某1层的计算量

In [ ]:
#计算量
flops = "Not Available"

#Conv层：
# input:长度为1的元组，元组中的元素类型为Tensor的size=[bs,num_channels_in,height_in,width_in]
# output:Tensor size=[bs,num_channels_out,height_out,width_out]
#Conv层:计算量=卷积核参数个数*输出size
if class_name.find("Conv") != -1 and hasattr(module, "weight"):
    #.item():获取标量tensor的元素值
    flops = (
        torch.prod(
            torch.LongTensor(list(module.weight.data.size()))) *
        torch.prod(
            torch.LongTensor(list(output.size())[2:]))).item()
#Linear层
#input:长度为1的元组，元组中的元素类型为Tensor的size=[bs,num_in]
#output:Tensor size=[bs,num_out]
#Linear层:计算量=输出的size*输入的num_in
elif isinstance(module, nn.Linear):
    flops = (torch.prod(torch.LongTensor(list(output.size()))) \
             * input[0].size(1)).item()

In [ ]:
#将每1层信息append到summary列表中
summary.append(
    ModuleDetails(
        name=layer_name,
        #不记录num_channel
        input_size=list(input[0].size()),
        output_size=list(output.size()),
        num_parameters=params,
        multiply_adds=flops)
)

## Example

**module.register_forward_hook(hook)：hook-钩子函数：**https://www.cnblogs.com/hellcat/p/8512090.html
**model.apply(add_hooks):apply函数：将add_hooks函数递归地应用到网络模型model的每1层module中**

In [ ]:
def add_hooks(module):

    def hook(module, input, output):
        #网络层module的类名
        class_name = str(module.__class__.__name__)

        instance_index = 1
        if class_name not in layer_instances:
            layer_instances[class_name] = instance_index
        else:
            instance_index = layer_instances[class_name] + 1
            layer_instances[class_name] = instance_index

        #网络层名字=网络层类名_出现的次数
        layer_name = class_name + "_" + str(instance_index)

        #参数数量
        params = 0
        #卷积层/Batch norm层/Linear层
        if class_name.find("Conv") != -1 or class_name.find("BatchNorm") != -1 or \
           class_name.find("Linear") != -1:
            #params：module.parameters()参数的数量
            for param_ in module.parameters():
                params += param_.view(-1).size(0)

        #计算量
        flops = "Not Available"
        #卷积层:计算量=卷积核参数个数*输出size
        if class_name.find("Conv") != -1 and hasattr(module, "weight"):
            #.item():获取标量tensor的元素值
            flops = (
                torch.prod(
                    torch.LongTensor(list(module.weight.data.size()))) *
                torch.prod(
                    torch.LongTensor(list(output.size())[2:]))).item()
        #Conv层：
        # input:长度为1的元组，元组中的元素类型为Tensor的size=[bs,num_channels_in,height_in,width_in]
        # output:Tensor size=[bs,num_channels_out,height_out,width_out]

        #Linear层
        #input:长度为1的元组，元组中的元素类型为Tensor的size=[bs,num_in]
        #output:Tensor size=[bs,num_out]
        elif isinstance(module, nn.Linear):
            flops = (torch.prod(torch.LongTensor(list(output.size()))) \
                     * input[0].size(1)).item()

        if isinstance(input[0], list):
            input = input[0]

        if isinstance(output, list):
            output = output[0]

        #将每1层信息append到summary列表中
        summary.append(
            ModuleDetails(
                name=layer_name,
                #不记录num_channel
                input_size=list(input[0].size()),
                output_size=list(output.size()),
                num_parameters=params,
                multiply_adds=flops)
        )

    #module是model的某一层：卷积层/Batch norm层/Linear层
    if not isinstance(module, nn.ModuleList) \
       and not isinstance(module, nn.Sequential) \
       and module != model:
        #module每次前向传播执行结束后会执行钩子函数(hook)
        #钩子函数不应修改输入和输出，并且在使用后应及时删除，以避免每次都运行钩子增加运行负载
        hooks.append(module.register_forward_hook(hook))

In [1]:
#model.eval()模式:不做batch norm和dropout
model.eval()
#apply(fn)：将fn函数递归地应用到网络模型的每1层中，主要用在参数的初始化。
model.apply(add_hooks)

space_len = item_length

model(*input_tensors)
for hook in hooks:
    hook.remove()
    
details=''
#总参数量
params_sum = 0
#总计算量
flops_sum = 0
for layer in summary:
    params_sum += layer.num_parameters
    if layer.multiply_adds != "Not Available":
        flops_sum += layer.multiply_adds
#os.linesep：终止符(换行)
details += os.linesep \
    + "Total Parameters: {:,}".format(params_sum) \
    + os.linesep + '-' * space_len * 5 + os.linesep
details += "Total Multiply Adds (For Convolution and Linear Layers only): {:,} GFLOPs".format(flops_sum/(1024**3)) \
    + os.linesep + '-' * space_len * 5 + os.linesep

details += "Number of Layers" + os.linesep
#记录每1类层出现的次数
for layer in layer_instances:
    details += "{} : {} layers   ".format(layer, layer_instances[layer])

NameError: name 'model' is not defined